### Examining the Batch Normalization method

    * From the paper "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift" by
      Sergey Ioffe and Christian Szegedy
      
    * Coding the method using custom layers in tensorflow 2.1.0 to understand the math and comparing against the API
    
    * Using MNIST data set

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.1.0'

In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Loading MNIST data

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


#### Checking min and max values of the input

In [6]:
np.max(x_train)

255

In [7]:
np.min(x_train)

0

#### Scaling the data using the Standard Scaler

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
stdscaler = StandardScaler(with_mean=True, with_std=True)

In [10]:
stdscaler_fit = stdscaler.fit(x_train.reshape(-1, 28*28))

In [11]:
stdscaler_fit.mean_.shape

(784,)

In [12]:
# average mean
np.sqrt(stdscaler_fit.mean_.mean())

5.772211140440891

In [13]:
stdscaler_fit.n_samples_seen_

60000

In [14]:
stdscaler_fit.scale_.shape

(784,)

In [15]:
# average std
np.sqrt(stdscaler_fit.var_.mean())

66.12879201995706

In [16]:
x_train_scaled =  stdscaler_fit.transform(x_train.reshape(-1,28*28)).reshape(-1,28,28)
x_train_scaled.shape

(60000, 28, 28)

In [17]:
x_test_scaled =  stdscaler_fit.transform(x_test.reshape(-1,28*28)).reshape(-1,28,28)
x_test_scaled.shape

(10000, 28, 28)

#### Checking the min and max values

In [18]:
np.min(x_train_scaled)

-1.2742078920822268

In [19]:
np.max(x_train_scaled)

244.94693302873063

In [20]:
np.var(x_train_scaled)

0.9145408163265558

In [21]:
np.mean(x_train_scaled)

-2.1974863349995617e-18

#### One-hot encoding the y labels

In [22]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))

    * The labels are more or less balanced

In [23]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009]))

In [24]:
y_train_coded =  tf.keras.utils.to_categorical(y_train, num_classes=10)
y_train_coded.shape

(60000, 10)

In [25]:
print("Label: ", y_train[0],'\n',"One-hot encoded: ", y_train_coded[0])

Label:  5 
 One-hot encoded:  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [26]:
y_test_coded =  tf.keras.utils.to_categorical(y_test, num_classes=10)
y_test_coded.shape

(10000, 10)

#### Creating validation data

In [27]:
from sklearn.utils import shuffle

In [31]:
# for reproducibility
random_seed = 100 

In [28]:
x_train_scaled.shape

(60000, 28, 28)

In [29]:
y_train_coded.shape

(60000, 10)

In [32]:
x_train_scaled, y_train_coded = shuffle(x_train_scaled, y_train_coded, random_state = random_seed) 

In [33]:
x_train_scaled.shape

(60000, 28, 28)

In [34]:
y_train_coded.shape

(60000, 10)

In [35]:
x_valid_scaled = x_train_scaled[:5000]
y_valid_coded = y_train_coded[:5000]

In [36]:
x_valid_scaled.shape

(5000, 28, 28)

In [37]:
y_valid_coded.shape

(5000, 10)

In [38]:
x_train_scaled = x_train_scaled[5000:]
y_train_coded = y_train_coded[5000:]

#### Creating a tf dataset for training on a Model with dense layers

In [39]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_scaled.reshape(-1, 784), y_train_coded))

In [40]:
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid_scaled.reshape(-1, 784), y_valid_coded))

In [41]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_scaled.reshape(-1,784), y_test_coded))

In [42]:
minibatch =  60 # in the paper

In [43]:
buffer_size = len(y_train)
buffer_size

60000

In [44]:
# shuffle first, batch 2nd, then prefetch
train_dataset = train_dataset.shuffle(buffer_size=buffer_size, seed=random_seed, 
                                      reshuffle_each_iteration=True).batch(batch_size=minibatch, 
                                                                           drop_remainder=True).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [45]:
valid_dataset = valid_dataset.shuffle(buffer_size=buffer_size, seed=random_seed, 
                                      reshuffle_each_iteration=False).batch(batch_size=minibatch).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [46]:
test_dataset = test_dataset.shuffle(buffer_size=buffer_size, seed=random_seed, 
                                      reshuffle_each_iteration=False).batch(batch_size=minibatch).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [47]:
for i in test_dataset.take(1):

    print(i[0][0].shape, '\n')
    print(i[1][0])

(784,) 

tf.Tensor([0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


### Building a Model using Dense Layers and Batch Norm from Keras

    * Using the model architecture from the paper

In [155]:
tf.keras.backend.clear_session()

In [156]:
input_layer = tf.keras.Input(shape = (784,), name = 'input')

In [157]:
units1, units2, units3 = 100,100,100

In [158]:
kerasdense1 = tf.keras.layers.Dense(units = units1, activation=None, 
                                    kernel_initializer=tf.keras.initializers.Orthogonal(gain=1,seed=random_seed))(input_layer)

In [52]:
# mean and variance are calculated for the minibatch.
# setting momentum = 0 for now. Population statistics will be calculated separately for inference.
# Initializing moving mean and variance using the statistics from the StandardScaler fit

# """bn_layer = tf.keras.layers.BatchNormalization(axis = [-1], momentum=0.0, epsilon=0.001, center=True, scale=True, 
#                                              beta_initializer = tf.keras.initializers.zeros(), 
#                                               gamma_initializer = tf.keras.initializers.ones(), 
#                                              moving_mean_initializer = tf.keras.initializers.zeros(), 
#                                              moving_variance_initializer = tf.keras.initializers.ones(), trainable=True)"""
# the "bn_layer" object can be used only once. Have to write a new batch_norm for every layer. Hence, will use a function

In [53]:
def bn_layer(axis = [-1]):
    
    """
    returns a batch norm layer
    
    Parameters:
    axis: list of integers. Default is [-1] which is confusing because the mean and averages are
    calculated across the minibatch rather than the features.   
    
    Anyways, will be cross-checking against custom code later
    
    """
    
    
    return tf.keras.layers.BatchNormalization(axis = axis, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                             beta_initializer = tf.keras.initializers.zeros(), 
                                              gamma_initializer = tf.keras.initializers.ones(), 
                                             moving_mean_initializer = tf.keras.initializers.zeros(), 
                                             moving_variance_initializer = tf.keras.initializers.ones(), trainable=True)

In [159]:
kerasdensebn1 = bn_layer(axis = [-1])(kerasdense1, training=True)

In [160]:
activation1 = tf.keras.layers.Activation(activation = tf.nn.tanh)(kerasdensebn1)

In [161]:
kerasdense2 = tf.keras.layers.Dense(units = units2, activation=None, 
                                    kernel_initializer=tf.keras.initializers.Orthogonal(gain=1,seed=random_seed))(activation1)
kerasdensebn2 = bn_layer(axis = [-1])(kerasdense2, training=True)
activation2 = tf.keras.layers.Activation(activation = tf.nn.tanh)(kerasdensebn2)

In [162]:
kerasdense3 = tf.keras.layers.Dense(units = units3, activation=None, 
                                    kernel_initializer=tf.keras.initializers.Orthogonal(gain=1,seed=random_seed))(activation2)
kerasdensebn3 = bn_layer(axis = [-1])(kerasdense3, training=True)
activation3 = tf.keras.layers.Activation(activation = tf.nn.tanh)(kerasdensebn3)

In [164]:
output_layer = tf.keras.layers.Dense(units = 10, activation=None,
                                    kernel_initializer=tf.keras.initializers.Orthogonal(gain=1,seed=random_seed))(activation3)
# no softmax activation

In [165]:
keras_fc_model = tf.keras.Model(inputs = [input_layer], outputs = [output_layer], name = 'kerasmodel1')

In [166]:
keras_fc_model.summary()

Model: "kerasmodel1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation_1 (Activation)    (None, 100)               

In [61]:
keras_fc_model.losses

[]

In [62]:
len(keras_fc_model.weights)

20

In [63]:
# batch norm layer variables
keras_fc_model.layers[2].weights

[<tf.Variable 'batch_normalization/gamma:0' shape=(100,) dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=float32)>,
 <tf.Variable 'batch_normalization/beta:0' shape=(100,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

#### Compiling

In [167]:
keras_fc_model.compile(optimizer =  tf.keras.optimizers.Adam(learning_rate=0.001), 
                      loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                      metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# USing tf.keras.metrics.AUC() give:

# """InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] 
# [x (kerasmodel1/dense_3/BiasAdd:0) = ] [[0.7673769 2.2196033 1.78565049...]...] [y (metrics/auc/Cast_1/x:0) = ] [0]
#	 [[{{node metrics/auc/assert_greater_equal/Assert/AssertGuard/else/_1/Assert}}]] [Op:__inference_distributed_function_103831]"""

# To use this metric, the predictions have to be between 0 and 1, implies softmax needs to be used.
                      
   

In [65]:
keras_fc_model.name

'kerasmodel1'

#### Callbacks

##### Learning rate
    * Adam optimizer can do adaptive learning rate feature

##### Not setting EarlyStopping

##### Save best model

In [66]:
import os

In [67]:
keras_savedmodels = 'keras_savedmodels'

if os.path.exists(keras_savedmodels):
    pass
else:
    os.mkdir(keras_savedmodels)

In [124]:
cb_savemodel = tf.keras.callbacks.ModelCheckpoint(os.path.join(keras_savedmodels, 'model_{epoch}-{val_loss:.3f}.h5'), mode = 'min',
                                                  monitor = 'val_loss',
                                                 verbose = 1, save_best_only=True)

##### Tensorboard

In [69]:
keras_models_logs = 'keras_models_logs'

if os.path.exists(keras_models_logs):
    pass
else:
    os.mkdir(keras_models_logs)

In [70]:
cb_tboard = tf.keras.callbacks.TensorBoard(log_dir=keras_models_logs,histogram_freq=1, write_graph=True,
                                          write_images=True)

In [136]:
cblist = [cb_savemodel, cb_tboard]

#### Fitting

In [104]:
epochs1=10

In [168]:
keras_history1 = keras_fc_model.fit(train_dataset, epochs=epochs1, verbose = 1, callbacks=cblist,
                                   validation_data=valid_dataset, shuffle = True)

Train for 916 steps, validate for 84 steps
Epoch 1/10
901/916 [============================>.] - ETA: 0s - loss: 0.3034 - categorical_accuracy: 0.9115
Epoch 00001: val_loss did not improve from -0.00001
916/916 [==============================] - 5s 5ms/step - loss: 0.3024 - categorical_accuracy: 0.9118 - val_loss: 0.2091 - val_categorical_accuracy: 0.9378
Epoch 2/10
906/916 [============================>.] - ETA: 0s - loss: 0.1611 - categorical_accuracy: 0.9523
Epoch 00002: val_loss did not improve from -0.00001
916/916 [==============================] - 4s 4ms/step - loss: 0.1609 - categorical_accuracy: 0.9524 - val_loss: 0.1649 - val_categorical_accuracy: 0.9484
Epoch 3/10
910/916 [============================>.] - ETA: 0s - loss: 0.1174 - categorical_accuracy: 0.9639
Epoch 00003: val_loss did not improve from -0.00001
916/916 [==============================] - 4s 4ms/step - loss: 0.1171 - categorical_accuracy: 0.9640 - val_loss: 0.1411 - val_categorical_accuracy: 0.9582
Epoch 4/10
9

In [141]:
tf.keras.backend.clear_session()

In [143]:
del keras_history1

In [142]:
del keras_fc_model

In [131]:
import gc

In [144]:
gc.collect()

14265